# Vanilla GAN (Generative Adversarial Network)

In [1]:
from torchvision.datasets import FashionMNIST
from torchvision.utils import make_grid
from torchvision import transforms

from torch.utils.data import DataLoader

import torch.optim as optim
import torch.nn as nn
import torch

import matplotlib.pyplot as plt
import numpy as np
import imageio

BATCH_SIZE = 100
LEAKY_ReLU_VALUE = 0.2
DROPOUT_VALUE = 0.3

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <44B645FB-F027-3EE5-86D7-DBF8E2FC6264> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [23]:
transform   = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
trainset    = FashionMNIST(root='../data/', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
device      = torch.device("cpu")

In [100]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.n_features = 128
        self.n_out = 784
        self.linear = nn.Sequential(
            nn.Linear(self.n_features, 256),        nn.LeakyReLU(LEAKY_ReLU_VALUE),
            nn.Linear(256, 512),                    nn.LeakyReLU(LEAKY_ReLU_VALUE),
            nn.Linear(512, 1024),                   nn.LeakyReLU(LEAKY_ReLU_VALUE),
            nn.Linear(1024, self.n_out),            nn.Tanh()
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.view(-1, 1, 28, 28)
        print(x.shape)
        return x

In [94]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.n_in = 784
        self.n_out = 1
        self.linear = nn.Sequential(
            nn.Linear(self.n_in, 1024),         nn.LeakyReLU(LEAKY_ReLU_VALUE), 
            nn.Dropout(DROPOUT_VALUE),
            nn.Linear(1024, 512),               nn.LeakyReLU(LEAKY_ReLU_VALUE),
            nn.Dropout(DROPOUT_VALUE),
            nn.Linear(512, 256),                nn.LeakyReLU(LEAKY_ReLU_VALUE),
            nn.Dropout(DROPOUT_VALUE),
            nn.Linear(256, self.n_out),         nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(-1, 784)
        print("Discrimiatorr forward!")
        print(x.shape)
        x = self.linear(x)
        return x

In [95]:
generator = Generator()
discriminator = Discriminator()

In [96]:
g_optim = optim.Adam(generator.parameters(), lr=2e-4)
d_optim = optim.Adam(discriminator.parameters(), lr=2e-4)

g_losses, d_losses = list(), list()
images = list()

# BCE (BinaryCrossEntropy)
criterion = nn.BCELoss()

In [97]:
def noise(n, n_features=128):
    data = torch.randn(n, n_features)
    return data

def label_ones(size):
    data = torch.ones(size, 1)
    return data.to(device)

def label_zeros(size):
    data = torch.zeros(size, 1)
    return data.to(device)

In [98]:
def train_discriminator(optimizer, dataReal, dataFake):
    n = dataReal.size(0)
    optimizer.zero_grad()

    predictionReal = discriminator(dataReal).to(device)
    print(predictionReal.shape)
    discriminatorLoss = criterion(predictionReal, label_ones(n)).to(device)

    predictionFake = generator(dataFake).to(device)
    print(predictionFake.shape)
    generatorLoss = criterion(predictionFake, label_zeros(n)).to(device)
    
    print(discriminatorLoss.shape, generatorLoss.shape)
    loss = discriminatorLoss + generatorLoss
    loss.backward()
    optimizer.step()
    return loss.item()

def train_generator(optimizer, dataFake):
    n = dataFake.size(0)
    optimizer.zero_grad()
    prediciton = discriminator(dataFake)
    
    loss = criterion(prediciton, label_ones(n))
    loss.backward()
    optimizer.step()
    return loss.item()

In [101]:
noiseTest = noise(64)
l = len(trainloader)
for epoch in range(151):
    generatorLoss, discriminatorLoss = 0.0, 0.0
    for data in trainloader:
        images, _ = data
        n = len(images)
        
        dataFake = generator(noise(n)).detach().to(device)
        dataReal = images.to(device)

        discriminatorLoss += train_discriminator(d_optim, dataReal, dataFake).to(device)
        dataFake = generator(noise(n))
        generatorLoss += train_generator(g_optim, dataFake)

    image = generator(noiseTest).detach().cpu()
    image = make_grid(image)
    images.append(image)

    g_losses.append(generatorLoss / l)
    d_losses.append(discriminatorLoss / l)
    print("Epoch : {}. GenertorLoss : {:.3f}\tDiscriminatorLoss : {:.3f}\r".format(epoch, generatorLoss / l, discriminatorLoss / l))

Discrimiatorr forward!
torch.Size([100, 784])
torch.Size([100, 1])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2800x28 and 128x256)